In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobaccesskey"))

In [0]:
subsciber_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/subscriber.csv", header=True, inferSchema=True)

In [0]:
#subsciber_data.show(10,False)

+----------+----------+-----------+-----------------+----------+------+--------------+-------+------------+--------+---------+--------+----------+----------+
|sub_id    |first_name|last_name  |Street           |Birth_date|Gender|Phone         |Country|City        |Zip Code|Subgrp_id|Elig_ind|eff_date  |term_date |
+----------+----------+-----------+-----------------+----------+------+--------------+-------+------------+--------+---------+--------+----------+----------+
|SUBID10000|Harbir    |Vishwakarma|Baria Marg       |1924-06-30|Female|+91 0112009318|India  |Rourkela    |767058  |S107     |Y       |1944-06-30|1954-01-14|
|SUBID10001|Brahmdev  |Sonkar     |Lala Marg        |1948-12-20|Female|+91 1727749552|India  |Tiruvottiyur|34639   |S105     |Y       |1968-12-20|1970-05-16|
|SUBID10002|Ujjawal   |Devi       |Mammen Zila      |1980-04-16|Male  |+91 8547451606|India  |Berhampur   |914455  |S106     |N       |2000-04-16|2008-05-04|
|SUBID10003|Ballari   |Mishra     |Sahni Zila       

In [0]:
#subsciber_data.select([count(when(col(c).isNull(), c)).alias(c) for c in subsciber_data.columns]).show()

+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+
|sub_id|first_name|last_name|Street|Birth_date|Gender|Phone|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+
|     0|        27|        0|     0|         0|     0|    3|      0|   0|       0|        2|       4|       0|        0|
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+



In [0]:
subsciber_data = subsciber_data.fillna({"Elig_ind":"N", "first_name":"Guest/NA"})

In [0]:
subsciber_data = subsciber_data.drop('Phone')

In [0]:
#subsciber_data.show(5,False)

+----------+----------+-----------+-----------+----------+------+-------+------------+--------+---------+--------+----------+----------+
|sub_id    |first_name|last_name  |Street     |Birth_date|Gender|Country|City        |Zip Code|Subgrp_id|Elig_ind|eff_date  |term_date |
+----------+----------+-----------+-----------+----------+------+-------+------------+--------+---------+--------+----------+----------+
|SUBID10000|Harbir    |Vishwakarma|Baria Marg |1924-06-30|Female|India  |Rourkela    |767058  |S107     |Y       |1944-06-30|1954-01-14|
|SUBID10001|Brahmdev  |Sonkar     |Lala Marg  |1948-12-20|Female|India  |Tiruvottiyur|34639   |S105     |Y       |1968-12-20|1970-05-16|
|SUBID10002|Ujjawal   |Devi       |Mammen Zila|1980-04-16|Male  |India  |Berhampur   |914455  |S106     |N       |2000-04-16|2008-05-04|
|SUBID10003|Ballari   |Mishra     |Sahni Zila |1969-09-25|Female|India  |Bihar Sharif|91481   |S104     |N       |1989-09-25|1995-06-05|
|SUBID10004|Devnath   |Srivastav  |Magar 

In [0]:
subsciber_data = subsciber_data.withColumn("subsciber_age", (months_between(current_date(), col('Birth_date'))/12).cast("int"))
subsciber_data = subsciber_data.drop(col("Birth_date"))

In [0]:
#subsciber_data.show(5,False)

+----------+----------+-----------+-----------+------+-------+------------+--------+---------+--------+----------+----------+-------------+
|sub_id    |first_name|last_name  |Street     |Gender|Country|City        |Zip Code|Subgrp_id|Elig_ind|eff_date  |term_date |subsciber_age|
+----------+----------+-----------+-----------+------+-------+------------+--------+---------+--------+----------+----------+-------------+
|SUBID10000|Harbir    |Vishwakarma|Baria Marg |Female|India  |Rourkela    |767058  |S107     |Y       |1944-06-30|1954-01-14|99           |
|SUBID10001|Brahmdev  |Sonkar     |Lala Marg  |Female|India  |Tiruvottiyur|34639   |S105     |Y       |1968-12-20|1970-05-16|74           |
|SUBID10002|Ujjawal   |Devi       |Mammen Zila|Male  |India  |Berhampur   |914455  |S106     |N       |2000-04-16|2008-05-04|43           |
|SUBID10003|Ballari   |Mishra     |Sahni Zila |Female|India  |Bihar Sharif|91481   |S104     |N       |1989-09-25|1995-06-05|54           |
|SUBID10004|Devnath 

In [0]:
#subsciber_data.select("*").where(col("Subgrp_id").isNull()).show(4)

+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+-------------+
|    sub_id|first_name|last_name|   Street|Gender|Country|    City|Zip Code|Subgrp_id|Elig_ind|  eff_date| term_date|subsciber_age|
+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+-------------+
|SUBID10022|   Prakash|      Rao|   Sachar|Female|  India|Kottayam|  180680|     NULL|       N|1943-09-15|1948-10-19|          100|
|SUBID10049|   Paridhi|    Yadav|Sant Path|Female|  India|Jabalpur|  883754|     NULL|       N|1979-03-27|1985-06-01|           64|
+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+-------------+



In [0]:
#the reason why we are hardcoding because we dont have Subgrp_id for 2 subscribtions
subsciber_data = subsciber_data.withColumn("Subgrp_id",when(col("sub_id") =='SUBID10022', 'S110').otherwise(col("Subgrp_id")))
subsciber_data = subsciber_data.withColumn("Subgrp_id",when(col("sub_id") =='SUBID10049', 'S107').otherwise(col("Subgrp_id")))

In [0]:
#subsciber_data.show(6,False)

+----------+----------+-----------+------------+------+-------+------------+--------+---------+--------+----------+----------+-------------+
|sub_id    |first_name|last_name  |Street      |Gender|Country|City        |Zip Code|Subgrp_id|Elig_ind|eff_date  |term_date |subsciber_age|
+----------+----------+-----------+------------+------+-------+------------+--------+---------+--------+----------+----------+-------------+
|SUBID10000|Harbir    |Vishwakarma|Baria Marg  |Female|India  |Rourkela    |767058  |S107     |Y       |1944-06-30|1954-01-14|99           |
|SUBID10001|Brahmdev  |Sonkar     |Lala Marg   |Female|India  |Tiruvottiyur|34639   |S105     |Y       |1968-12-20|1970-05-16|74           |
|SUBID10002|Ujjawal   |Devi       |Mammen Zila |Male  |India  |Berhampur   |914455  |S106     |N       |2000-04-16|2008-05-04|43           |
|SUBID10003|Ballari   |Mishra     |Sahni Zila  |Female|India  |Bihar Sharif|91481   |S104     |N       |1989-09-25|1995-06-05|54           |
|SUBID10004|D

In [0]:
#subsciber_data.select([count(when(col(c).isNull(), c)).alias(c) for c in subsciber_data.columns]).show()

+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+-------------+
|sub_id|first_name|last_name|Street|Gender|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|subsciber_age|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+-------------+
|     0|         0|        0|     0|     0|      0|   0|       0|        0|       0|       0|        0|            0|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+-------------+



In [0]:
output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
subsciber_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/subsciberdatastagging.csv" % output_container_path)

True